# Analysis of `trips-info.xml`

In [ ]:
import pprint
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np

# normal = "berlin-1pct"
# normal = "../1pct-output-test"
# normal = "../simulation/charlottenburg/output"
normal = "../aws/N/output"

scenarios = [
  "apats-output"
#   "greater-berlin-1pct"
  # "../simulation/charlottenburg/output"
]

In [ ]:
def get_trips(file_path):
    trips = []
    
    for event, element in etree.iterparse(file_path, tag="tripinfo"):
        current_trip = {
            "id": element.attrib["id"],
            "duration": float(element.attrib["duration"]),
            "routeLength": float(element.attrib["routeLength"]),
            "rerouteNo": float(element.attrib["rerouteNo"])
        }
        
        emissions = element.getchildren()[0]
        for key in emissions.attrib:
            current_trip[key] = float(emissions.attrib[key])

        trips.append(current_trip)

        element.clear()

    return trips

metrics = {
    "distance": {
        "key": "routeLength",
        "scale": 1000,
        "unit": "km",
        "label": "Distance"
    },
    "duration": {
        "key": "duration",
        "scale": 60,
        "unit": "min",
        "label": "Duration"
    },
    "CO2": {
        "key": "CO2_abs",
        "unit": "mg",
        "label": "CO2"
    },
    "PMx": {
        "key": "PMx_abs",
        "unit": "mg",
        "label": "PMx"
    },
    "NOx": {
        "key": "NOx_abs",
        "unit": "mg",
        "label": "NOx"
    },
    "fuel": {
        "key": "fuel_abs",
        "unit": "ml",
        "label": "Fuel"
    },
}

# prefix n_ for normal
n_trips = get_trips(f"{normal}/trips-info.xml")
scenario_data = {}

for s in scenarios:
    trips = get_trips(f"{s}/trips-info.xml")
    scenario_data[s] = trips

### Total Amount of Trips

In [ ]:
print("Normal:", len(n_trips))

for s in scenario_data:
    print(f"{s}:", len(scenario_data[s]))

### Average Amount of Trips per Person

In [ ]:
def get_average_trips_per_person(trips):
    persons = {}
    for trip in trips:
        person, counter = trip["id"].split("_")
        if person in persons:
            persons[person] += 1
        else:
            persons[person] = 1

    return format(sum(persons.values()) / len(persons), ".3f")

print("Normal:", get_average_trips_per_person(n_trips), "trips")

for s in scenario_data:
    print(f"{s}:", get_average_trips_per_person(scenario_data[s]), "trips")

### Mean value comparison for all vehicles

In [ ]:
for metric in metrics:
    key = metrics[metric]["key"]
    unit = metrics[metric]["unit"]

    n_values = np.array(list(map(lambda t: t[key], n_trips)))
    if "scale" in metrics[metric]:
        scale = metrics[metric]["scale"]
        n_values = n_values / scale
    n_mean = n_values.mean()
    
    print(f"Mean {metric}")
    print(f"{normal}:", format(n_mean, '.3f'), unit)
    

    if len(scenarios) > 0:
        for s in scenarios:
            s_trips = scenario_data[s]
            s_values = np.array(list(map(lambda t: t[key], s_trips)))
            if "scale" in metrics[metric]:
                scale = metrics[metric]["scale"]
                s_values = s_values / scale
            s_mean = s_values.mean()

            print(f"{s}:", format(s_mean, '.3f'), unit)
            print("Absolute difference:", format(s_mean - n_mean, '.3f'), unit)
            print("Relative difference:", format((s_mean - n_mean) / n_mean, '.3f'), "%")
            print()
    else:
        print()

### Mean distance/duration comparison only for rerouted vehicles

In [ ]:
for s in scenarios:
    s_trips = scenario_data[s]

    rr_s_trips = list(filter(lambda t: t["rerouteNo"] > 0, s_trips))
    rr_ids = list(map(lambda t: t["id"], rr_s_trips))
    rr_n_trips = list(filter(lambda t: t["id"] in rr_ids, n_trips))

    for metric in metrics:
        key = metrics[metric]["key"]
        unit = metrics[metric]["unit"]

        n_values = np.array(list(map(lambda t: t[key], rr_n_trips)))
        s_values = np.array(list(map(lambda t: t[key], rr_s_trips)))

        if "scale" in metrics[metric]:
            scale = metrics[metric]["scale"]
            n_values = n_values / scale
            s_values = s_values / scale

        print(f"Mean {metric}")
        print(f"{normal}:", n_values.mean(), unit)
        print(f"{s}:", s_values.mean(), unit)
        print("Difference:", s_values.mean() - n_values.mean(), unit, "\n")

### Additional distance/duration for rerouted vehicles

In [ ]:
for s in scenarios:
    s_trips = scenario_data[s]

    rr_s_trips = list(filter(lambda t: t["rerouteNo"] > 0, s_trips))
    rr_ids = list(map(lambda t: t["id"], rr_s_trips))
    rr_n_trips = list(filter(lambda t: t["id"] in rr_ids, n_trips))

    for metric in metrics:
        key = metrics[metric]["key"]
        unit = metrics[metric]["unit"]

        additional_values = []
        additional_percent = []
        for rr_s_t in rr_s_trips:
            rr_n_t = list(filter(lambda t: t["id"] == rr_s_t["id"], rr_n_trips))[0]
            if rr_n_t[key] == 0:
                print(rr_s_t)
                continue
            additional_values.append(rr_s_t[key] - rr_n_t[key])
            additional_percent.append((rr_s_t[key] - rr_n_t[key]) / rr_n_t[key])
        
        additional_values = np.array(additional_values)
        additional_percent = np.array(additional_percent)

        if "scale" in metrics[metric]:
            scale = metrics[metric]["scale"]
            additional_values = additional_values / scale

        print(f"Mean additional {metric}:", format(additional_values.mean(), ".3f"), unit)
        print(f"Mean fractional increase:", format(additional_percent.mean(), ".3f"), "\n")